In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!source /content/drive/MyDrive/colab_env/bin/activate

In [ ]:
! pip install pyet swifter

In [4]:
import pandas as pd
import pyarrow
import pyet
import math
import datetime
import swifter
import numpy as np
import glob
import os
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
from scipy import stats

In [5]:
prism_data_daily = pd.read_parquet('/content/drive/MyDrive/EC_Tower/merged_prism_data_par_for_ameri_stations.parquet', engine='pyarrow')


In [6]:
# convert lat to radian
prism_data_daily["latitude_in_radian"] = list(map(math.radians, prism_data_daily["latitude"]))

prism_data_daily['time'] = pd.to_datetime(prism_data_daily['date'],format= "%Y-%m-%d" )

prism_data_daily.set_index('time', inplace=True)
prism_data_daily

,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,tmin,tmax,latitude,longitude,Site_ID,General_classification,Elevation,Energy_balance,Land_cover_details,Land_cover_type,latitude_in_radian
time,,,,,,,,,,,,,,,,,,,,
2008-12-31,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,4.565,6.442,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-01,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,4.836,7.053,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-02,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,4.656,7.567,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-03,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,0.441,9.665,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
2009-01-04,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,0.316,8.929,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards,0.636324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-26,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,6.817,16.126,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands,0.666726
2020-12-27,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,4.567,14.088,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands,0.666726
2020-12-28,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,4.429,13.631,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands,0.666726


In [7]:
# Solar Constant [ MJ m-2 min-1]
SOLAR_CONSTANT = 0.0820

def standard_date_to_Julian_day (
    standard_date_in_gregorian : str
    ) -> int:

    """
    Description
    -----------
    calculate Julian Day with standard date
    Ref:https://rafatieppo.github.io/post/2018_12_01_juliandate/
    ----------

    standard_date_in_gregorian : str
        Date with the specified standard

    Returns
    -------
    Julian Day : int
        Number of days of the year taking into account the leap year
    """

    fmt='%Y-%m-%d'
    sdtdate = datetime.datetime.strptime(standard_date_in_gregorian, fmt)
    sdtdate = sdtdate.timetuple()
    jdate = sdtdate.tm_yday

    return(jdate)


def inverse_relative_distance_earth_sun(
    julian_date: int
) -> float:

    """
    Description
    -----------
    calculate Inverse Relative Distance Earth Sun With Julian Date - eq 23 FAO56
    ----------


    Julian Day : int
        Number of days of the year taking into account the leap year

    Returns
    -------
    inverse_relative_distance_earth_sun : float
        inverse relative distance earth sun in Radian
    """

    return 1 + (0.033 * np.cos((2 * np.pi * julian_date)/365))

def solar_declination(
    julian_date: int
) -> float:

    """
    Description
    -----------
    calculate Solar Declination With Julian Date - eq 24 FAO56
    ----------

    julian_date : int
        Number of days of the year taking into account the leap year

    Returns
    -------
    solar_declination : float
        solar_declination in Radian
    """

    return 0.409 * np.sin(((2 * np.pi * julian_date) / 365) - 1.39)

def sunset_hour_angle(
    latitude : float,
    solar_declination : float
) -> float:

    """
    Description
    -----------
    calculate Sunset Hour Angle With Latitude and Solar Declination - eq 25 FAO56
    ----------

    latitude: float
        latitude in Radian
    solar_declination : float
        solar_declination in Radian

    Returns
    -------
    sunset_hour_angle : float
        sunset hour angle in Radian
    """

    return np.arccos(-np.tan(latitude) * np.tan(solar_declination))

def extraterrestrial_radiation(
    inverse_relative_distance_earth_sun : float,
    sunset_hour_angle : float,
    latitude : float,
    solar_declination : float
) -> float:

    """
    Description
    -----------
    calculate Extraterrestrial Radiation - eq 28 FAO56
    ----------

    inverse_relative_distance_earth_sun : float
        inverse relative distance earth sun in Radian
    sunset_hour_angle : float
        sunset hour angle in Radian
    latitude: float
        latitude in Radian
    solar_declination : float
        solar_declination in Radian

    Returns
    -------
    extraterrestrial_radiation : float
            extraterrestrial radiation in MJ/m**2/day
    """

    temp_1 = ((24 * 60 )/ np.pi) * SOLAR_CONSTANT * inverse_relative_distance_earth_sun

    temp_2 = sunset_hour_angle * np.sin(latitude) * np.sin(solar_declination)

    temp_3 = np.cos(latitude)* np.cos(solar_declination) * np.sin(sunset_hour_angle)

    return temp_1 * (temp_2 + temp_3)

def solar_or_Short_wave_Radiation_Hargreaves(
    extraterrestrial_radiation : float,
    tmax : float,
    tmin : float,
    Adjustment_coefficient_or_K_RS : float
)-> float:

    """
    Description
    -----------
    calculate Solar or shortwave radiation with max and min Tempreture - eq 50 FAO56
    ----------

    extraterrestrial_radiation : float
        extraterrestrial radiation in MJ/m**2/day
    tmax : float
        Maximum Temperature in celsius
    tmin : float
        Minimum Temperature in celsius
    Adjustment_coefficient_or_K_RS : float
        Adjustment coefficient in C**-0.5 -- between 0.16 to 0.19
            for interior locations, where land mass dominates and air masses are not strongly
            influenced by a large water body, kRs ≅ 0.16;
            for coastal locations, situated on or adjacent to the coast of a large land mass and where
            air masses are influenced by a nearby water body, kRs ≅ 0.19

    Returns
    -------
    Solar or shortwave radiation : float
        Solar or shortwave radiation in MJ/m**2/day
    """
    temp_1 = extraterrestrial_radiation


    return Adjustment_coefficient_or_K_RS * np.sqrt(tmax - tmin) * temp_1

In [8]:
Adjustment_coefficient_or_K_RS = 0.175


prism_data_daily['julian_day'] = prism_data_daily.apply(lambda row: standard_date_to_Julian_day(row['date']), axis=1)
prism_data_daily['dr'] = prism_data_daily.apply(lambda row: inverse_relative_distance_earth_sun(row['julian_day']), axis=1)
prism_data_daily['delta'] = prism_data_daily.apply(lambda row: solar_declination(row['julian_day']), axis=1)
prism_data_daily['omega_s'] = prism_data_daily.apply(lambda row: sunset_hour_angle(row['latitude_in_radian'], row["delta"]), axis=1)
prism_data_daily['ra'] = prism_data_daily.apply(lambda row: extraterrestrial_radiation(row['dr'], row["omega_s"], row["latitude_in_radian"], row["delta"]), axis=1)
prism_data_daily['rs'] = prism_data_daily.apply(lambda row: solar_or_Short_wave_Radiation_Hargreaves(row['ra'], row["tmax"], row["tmin"], Adjustment_coefficient_or_K_RS), axis=1)


prism_data_daily

,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,tmin,...,Energy_balance,Land_cover_details,Land_cover_type,latitude_in_radian,julian_day,dr,delta,omega_s,ra,rs
time,,,,,,,,,,,,,,,,,,,,,
2008-12-31,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,4.565,...,1.00,Peach,Orchards,0.636324,366,1.032995,-0.401008,1.252175,16.025685,3.842260
2009-01-01,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,4.836,...,1.00,Peach,Orchards,0.636324,1,1.032995,-0.401008,1.252175,16.025685,4.175779
2009-01-02,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,4.656,...,1.00,Peach,Orchards,0.636324,2,1.032980,-0.399564,1.253500,16.080602,4.801329
2009-01-03,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,0.441,...,1.00,Peach,Orchards,0.636324,3,1.032956,-0.398001,1.254930,16.139901,8.578247
2009-01-04,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,0.316,...,1.00,Peach,Orchards,0.636324,4,1.032922,-0.396320,1.256466,16.203566,8.321965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-26,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,6.817,...,0.86,Brackish tidal marsh,Wetlands,0.666726,361,1.032922,-0.406440,1.225258,14.737636,7.868961
2020-12-27,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,4.567,...,0.86,Brackish tidal marsh,Wetlands,0.666726,362,1.032956,-0.405594,1.226097,14.770615,7.975867
2020-12-28,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,4.429,...,0.86,Brackish tidal marsh,Wetlands,0.666726,363,1.032980,-0.404627,1.227054,14.808078,7.861001


In [9]:
# prism_data_daily["blaney_criddle"] = pyet.blaney_criddle(
#     tmean = prism_data_daily["tmean"],
#     lat = prism_data_daily["latitude_in_radian"]
# )

# prism_data_daily["hamon"] = pyet.hamon(
#     tmean = prism_data_daily["tmean"],
#     lat = prism_data_daily["latitude_in_radian"]
# )

prism_data_daily["oudin"] = pyet.oudin(
    tmean = prism_data_daily["tmean"],
    lat = prism_data_daily["latitude_in_radian"]
)

prism_data_daily["hargreaves"] = pyet.hargreaves(
    tmean = prism_data_daily["tmean"],
    tmax = prism_data_daily["tmax"],
    tmin = prism_data_daily["tmin"],
    lat = prism_data_daily["latitude_in_radian"]
)

prism_data_daily["abtew"] = pyet.abtew(
    tmean = prism_data_daily["tmean"],
    rs = prism_data_daily["rs"]
)


prism_data_daily["mcguinness_bordne"] = pyet.mcguinness_bordne(
    tmean = prism_data_daily["tmean"],
    lat = prism_data_daily["latitude_in_radian"]
)

# prism_data_daily["jensen_haise"] = pyet.jensen_haise(
#     tmean = prism_data_daily["tmean"],
#     lat = prism_data_daily["latitude_in_radian"]
# )
prism_data_daily = prism_data_daily.reset_index()

In [10]:
# Specify the directory path and file pattern

file_path = '/content/drive/MyDrive/EC_Tower/stations/*.csv'
csv_files = glob.glob(file_path)

dfs = []

for file in csv_files:

    ts_data = pd.read_csv(file)

    ts_data["Site_ID"] = file.removesuffix("_daily_data.csv")[41:]

    dfs.append(ts_data)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df



,date,rh,H,sw_in,t_dew,vp,theta_1,lw_out,LE,wd,...,ET_gap,ET_fill_val,Site_ID,H_qc_flag,Rn_qc_flag,LE_qc_flag,t_avg_qc_flag,vpd_qc_flag,G_qc_flag,ppt
0,2012-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,US-Tw2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,US-Tw2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,US-Tw2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,US-Tw2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,US-Tw2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20138,2018-05-31,58.591875,74.299147,346.442861,11.252931,1.361338,26.386760,404.013732,60.627478,242.604626,...,False,NaN,US-Tw3,NaN,NaN,NaN,NaN,NaN,NaN,0.0
20139,2018-06-01,54.740417,57.072189,354.218242,12.342688,1.462196,26.324656,420.252554,75.335373,282.614125,...,False,NaN,US-Tw3,NaN,NaN,NaN,NaN,NaN,NaN,0.0
20140,2018-06-02,45.873125,34.105066,356.110903,12.915099,1.533126,26.316750,438.610560,109.118007,238.779884,...,False,NaN,US-Tw3,NaN,NaN,NaN,NaN,NaN,NaN,0.0
20141,2018-06-03,47.508958,15.475881,353.838460,13.621733,1.604567,26.276646,443.144378,122.170050,225.033641,...,False,NaN,US-Tw3,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [11]:

# Merge the DataFrames
result = pd.merge(prism_data_daily, merged_df[['date', 'Site_ID', 'ET_fill']],
                  on=['date', 'Site_ID'],
                  how='left')

result

,time,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,...,dr,delta,omega_s,ra,rs,oudin,hargreaves,abtew,mcguinness_bordne,ET_fill
0,2008-12-31,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,...,1.032995,-0.401008,1.252175,16.025685,3.842260,0.676516,0.472973,0.818486,0.994479,NaN
1,2009-01-01,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,...,1.032995,-0.401008,1.252175,16.025685,4.175779,0.705217,0.523976,0.889905,1.036669,NaN
2,2009-01-02,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,...,1.032980,-0.399564,1.253500,16.080602,4.801329,0.718546,0.606803,1.023379,1.056262,NaN
3,2009-01-03,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,...,1.032956,-0.398001,1.254930,16.139901,8.578247,0.651802,1.035083,1.826573,0.958149,NaN
4,2009-01-04,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,...,1.032922,-0.396320,1.256466,16.203566,8.321965,0.626125,0.984862,1.771280,0.920404,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56974,2020-12-26,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,...,1.032922,-0.406440,1.225258,14.737636,7.868961,0.981236,1.223674,1.685809,1.442417,NaN
56975,2020-12-27,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,...,1.032956,-0.405594,1.226097,14.770615,7.975867,0.853682,1.147109,1.705223,1.254913,NaN
56976,2020-12-28,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,...,1.032980,-0.404627,1.227054,14.808078,7.861001,0.837833,1.117869,1.680189,1.231615,NaN
56977,2020-12-29,12_20201230,2020-12-29,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",-1.101,9.277900,15.402,0.592,...,1.032995,-0.403540,1.228128,14.850017,10.391113,0.855260,1.491673,2.221492,1.257232,NaN


In [12]:
def budyko(pet, p):
    # from Budyko 1958
    # p,pet = X
    if p == 0:
        x = 0
    else:
        # x = (1 + (0.5 * (pet / p)** (-1.5)))**(-1)
        x = math.sqrt(((pet/p)*math.tanh(p/pet))*(1-math.exp(-pet/p)))
    return x


def aet(ratio, p):
  return ratio * p

In [13]:


result['ratio_budyko_oudin'] = result.apply(lambda row: budyko(row['oudin'], row["ppt"]), axis=1)

result['aet_budyko_oudin'] = result.apply(lambda row: aet(row['ratio_budyko_oudin'], row["ppt"]), axis=1)


result['ratio_budyko_hargreaves'] = result.apply(lambda row: budyko(row['hargreaves'], row["ppt"]), axis=1)

result['aet_budyko_hargreaves'] = result.apply(lambda row: aet(row['ratio_budyko_hargreaves'], row["ppt"]), axis=1)


result['ratio_budyko_abtew'] = result.apply(lambda row: budyko(row['abtew'], row["ppt"]), axis=1)

result['aet_budyko_abtew'] = result.apply(lambda row: aet(row['ratio_budyko_abtew'], row["ppt"]), axis=1)


result['ratio_budyko_mcguinness_bordne'] = result.apply(lambda row: budyko(row['mcguinness_bordne'], row["ppt"]), axis=1)

result['aet_budyko_mcguinness_bordne'] = result.apply(lambda row: aet(row['ratio_budyko_mcguinness_bordne'], row["ppt"]), axis=1)


result




,time,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,...,mcguinness_bordne,ET_fill,ratio_budyko_oudin,aet_budyko_oudin,ratio_budyko_hargreaves,aet_budyko_hargreaves,ratio_budyko_abtew,aet_budyko_abtew,ratio_budyko_mcguinness_bordne,aet_budyko_mcguinness_bordne
0,2008-12-31,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,...,0.994479,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2009-01-01,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,...,1.036669,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2009-01-02,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,...,1.056262,NaN,0.233268,0.676011,0.198876,0.576341,0.323011,0.936085,0.332279,0.962946
3,2009-01-03,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,...,0.958149,NaN,0.951844,0.217020,0.986726,0.224974,0.997250,0.227373,0.983290,0.224190
4,2009-01-04,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,...,0.920404,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56974,2020-12-26,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,...,1.442417,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
56975,2020-12-27,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,...,1.254913,NaN,0.558132,0.684828,0.669477,0.821448,0.802162,0.984252,0.701850,0.861170
56976,2020-12-28,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,...,1.231615,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
56977,2020-12-29,12_20201230,2020-12-29,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",-1.101,9.277900,15.402,0.592,...,1.257232,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
# Assuming your DataFrame is named 'df' and the two columns are 'actual' and 'predicted'
def calculate_metrics(df, actual_col, predicted_col):
    df = df.dropna()
    actual = df[actual_col]
    predicted = df[predicted_col]

    rmse = np.sqrt(mean_squared_error(actual, predicted))

    nse = 1 - (np.sum((actual - predicted) ** 2) / np.sum((actual - np.mean(actual)) ** 2))

    nrmse = rmse / (np.max(actual) - np.min(actual))

    r = stats.pearsonr(predicted,actual)[0]
    r2= r**2

    return rmse, nse, nrmse, r2

In [16]:
filtered_df = result[result['ET_fill'].apply(lambda x: isinstance(x, float) and not np.isnan(x))]

filtered_df = filtered_df[filtered_df['ratio_budyko_hargreaves'].apply(lambda x: isinstance(x, float) and not np.isnan(x))]

filtered_df = filtered_df[filtered_df['ratio_budyko_hargreaves'].apply(lambda x: isinstance(x, float) and x != 0)]

In [17]:
observed_col = 'ET_fill'
method_cols = [col for col in filtered_df.columns if col.startswith('aet_')]


In [18]:
# Function to calculate metrics for a specific time scale
def calculate_timescale_metrics(data, timescale):
    results = []
    for method in method_cols:
        metrics = data.groupby('Site_ID').apply(lambda x: calculate_metrics(x, observed_col, method))
        metrics = metrics.reset_index()
        metrics['Method'] = method
        metrics['Timescale'] = timescale
        results.append(metrics)
    return pd.concat(results, ignore_index=True)



# Daily calculations
daily_metrics = calculate_timescale_metrics(filtered_df, 'Daily')


In [19]:
# Monthly calculations
filtered_df['year_month'] = filtered_df['time'].dt.strftime('%Y-%m')
monthly_data = filtered_df.groupby(['Site_ID', 'year_month']).agg({observed_col: 'sum', **{col: 'sum' for col in method_cols}}).reset_index()
monthly_metrics = calculate_timescale_metrics(monthly_data, 'Monthly')


In [20]:
# Yearly calculations
filtered_df['year'] = filtered_df['time'].dt.year
yearly_data = filtered_df.groupby(['Site_ID', 'year']).agg({observed_col: 'sum', **{col: 'sum' for col in method_cols}}).reset_index()
yearly_metrics = calculate_timescale_metrics(yearly_data, 'Yearly')

In [21]:
# Combine all metrics
all_metrics = pd.concat([daily_metrics, monthly_metrics, yearly_metrics], ignore_index=True)
all_metrics

,Site_ID,0,Method,Timescale
0,Almond_High,"(2.116681665853018, -0.38650325587784384, 0.24...",aet_budyko_oudin,Daily
1,Almond_Low,"(2.206357829641578, -0.33709768500795034, 0.23...",aet_budyko_oudin,Daily
2,Almond_Med,"(1.979526877128931, -0.2926781289992806, 0.209...",aet_budyko_oudin,Daily
3,MB_Pch,"(1.3241453697079733, 0.006773678836961672, 0.1...",aet_budyko_oudin,Daily
4,RIP760,"(0.970194074120671, 0.0814374429468232, 0.1724...",aet_budyko_oudin,Daily
...,...,...,...,...
151,US-Sne,"(34.66160106294968, 0.5759713523439812, 0.2339...",aet_budyko_mcguinness_bordne,Yearly
152,US-Srr,"(7.671218909196558, 0.7829225035341012, 0.1752...",aet_budyko_mcguinness_bordne,Yearly
153,US-Tw2,"(24.20803389875078, -0.12376190185424774, 0.53...",aet_budyko_mcguinness_bordne,Yearly
154,US-Tw3,"(9.078033282896502, 0.9050762492526264, 0.1018...",aet_budyko_mcguinness_bordne,Yearly


In [22]:

# Create separate columns for each metric
all_metrics[['rmse', 'nse', 'nrmse', 'r2']] = pd.DataFrame(all_metrics[0].tolist(), index=all_metrics.index)

# Drop the original tuple column
all_metrics = all_metrics.drop(0, axis=1)
all_metrics

,Site_ID,Method,Timescale,rmse,nse,nrmse,r2
0,Almond_High,aet_budyko_oudin,Daily,2.116682,-0.386503,0.246681,0.217842
1,Almond_Low,aet_budyko_oudin,Daily,2.206358,-0.337098,0.234130,0.091353
2,Almond_Med,aet_budyko_oudin,Daily,1.979527,-0.292678,0.209756,0.190878
3,MB_Pch,aet_budyko_oudin,Daily,1.324145,0.006774,0.191953,0.170442
4,RIP760,aet_budyko_oudin,Daily,0.970194,0.081437,0.172473,0.149782
...,...,...,...,...,...,...,...
151,US-Sne,aet_budyko_mcguinness_bordne,Yearly,34.661601,0.575971,0.233971,0.940077
152,US-Srr,aet_budyko_mcguinness_bordne,Yearly,7.671219,0.782923,0.175277,0.979144
153,US-Tw2,aet_budyko_mcguinness_bordne,Yearly,24.208034,-0.123762,0.530038,1.000000
154,US-Tw3,aet_budyko_mcguinness_bordne,Yearly,9.078033,0.905076,0.101882,0.961373


In [23]:
# Reshape the data for easier analysis
all_metrics_wide = all_metrics.pivot_table(
    index=['Site_ID', 'Method'],
    columns='Timescale',
    values=['rmse', 'nse', 'nrmse', 'r2']
).reset_index()

all_metrics_wide

Site_ID                        Method     nrmse            \
Timescale                                                Daily   Monthly   
0          Almond_High              aet_budyko_abtew  0.229724  0.193346   
1          Almond_High         aet_budyko_hargreaves  0.237406  0.224495   
2          Almond_High  aet_budyko_mcguinness_bordne  0.229751  0.202378   
3          Almond_High              aet_budyko_oudin  0.246681  0.246279   
4           Almond_Low              aet_budyko_abtew  0.222837  0.209191   
5           Almond_Low         aet_budyko_hargreaves  0.226222  0.237372   
6           Almond_Low  aet_budyko_mcguinness_bordne  0.223324  0.215607   
7           Almond_Low              aet_budyko_oudin  0.234130  0.262698   
8           Almond_Med              aet_budyko_abtew  0.198860  0.192854   
9           Almond_Med         aet_budyko_hargreaves  0.202895  0.221943   
10          Almond_Med  aet_budyko_mcguinness_bordne  0.198148  0.200338   
11          Almond_Med              aet_budyko_oudin  0.209756  0.245028   
12              MB_Pch              aet_budyko_abtew  0.210476  0.203406   
13              MB_Pch         aet_budyko_hargreaves  0.191718  0.183443   
14              MB_Pch  aet_budyko_mcguinness_bordne  0.199599  0.185698   
15              MB_Pch              aet_budyko_oudin  0.191953  0.191463   
16              RIP760              aet_budyko_abtew  0.226792  0.174658   
17              RIP760         aet_budyko_hargreaves  0.183861  0.120602   
18              RIP760  aet_budyko_mcguinness_bordne  0.206164  0.147697   
19              RIP760              aet_budyko_oudin  0.172473  0.125916   
20              SLM001              aet_budyko_abtew  0.300158  0.417997   
21              SLM001         aet_budyko_hargreaves  0.229007  0.270133   
22              SLM001  aet_budyko_mcguinness_bordne  0.307664  0.423343   
23              SLM001              aet_budyko_oudin  0.218189  0.249275   
24              US-Bi1              aet_budyko_abtew  0.162243  0.114730   
25              US-Bi1         aet_budyko_hargreaves  0.152451  0.147506   
26              US-Bi1  aet_budyko_mcguinness_bordne  0.167045  0.121078   
27              US-Bi1              aet_budyko_oudin  0.158874  0.169888   
28              US-Bi2              aet_budyko_abtew  0.216478  0.105078   
29              US-Bi2         aet_budyko_hargreaves  0.189853  0.124209   
30              US-Bi2  aet_budyko_mcguinness_bordne  0.216533  0.105862   
31              US-Bi2              aet_budyko_oudin  0.191514  0.148014   
32              US-Sne              aet_budyko_abtew  0.245375  0.176783   
33              US-Sne         aet_budyko_hargreaves  0.241142  0.201672   
34              US-Sne  aet_budyko_mcguinness_bordne  0.250757  0.175039   
35              US-Sne              aet_budyko_oudin  0.245218  0.209005   
36              US-Srr              aet_budyko_abtew  0.224376  0.212158   
37              US-Srr         aet_budyko_hargreaves  0.201334  0.181293   
38              US-Srr  aet_budyko_mcguinness_bordne  0.234005  0.219223   
39              US-Srr              aet_budyko_oudin  0.206344  0.193992   
40              US-Tw2              aet_budyko_abtew  0.336913  0.279411   
41              US-Tw2         aet_budyko_hargreaves  0.317152  0.265707   
42              US-Tw2  aet_budyko_mcguinness_bordne  0.345583  0.271043   
43              US-Tw2              aet_budyko_oudin  0.321619  0.281169   
44              US-Tw3              aet_budyko_abtew  0.243971  0.153433   
45              US-Tw3         aet_budyko_hargreaves  0.229179  0.159895   
46              US-Tw3  aet_budyko_mcguinness_bordne  0.255966  0.160614   
47              US-Tw3              aet_budyko_oudin  0.233382  0.167062   
48              US-Twt              aet_budyko_abtew  0.163311  0.154888   
49              US-Twt         aet_budyko_hargreaves  0.147108  0.137686   
50              US-Twt  aet_budyko_mcguinness_bordne  0.167557 